In [2]:
import pandas as pd
import requests

## Get Teams

In [ ]:
url = "https://www.balldontlie.io/api/v1/teams"
r=requests.get(url)
data = r.json()
teams_df = pd.DataFrame(data['data'])
teams_df.to_csv("teams.csv")

## Get Players

In [3]:
players_url = "https://www.balldontlie.io/api/v1/players"
players_list = [] # create an empty df
def get_page(number):
    param = {"page" : number, "per_page" : 100} # define params based off of page number
    r=requests.get(players_url, params = param)
    # if the status is successful, we want to gather and clean the data
    if r.status_code == 200:
        # gather data
        data = r.json()
        df = pd.DataFrame(data['data'])
        # clean to extract data from within inner dictionary
        df['team_id'] = df['team'].apply(lambda x: x.get('id') if isinstance(x, dict) else None)
        df['team_abr'] = df['team'].apply(lambda x: x.get('abbreviation') if isinstance(x, dict) else None)
        df['city'] = df['team'].apply(lambda x: x.get('city') if isinstance(x, dict) else None)
        df = df.drop('team', axis=1)
        # now we want to save to the list
        players_list.append(df)
    # if the status gives an error, we would like to know what the error is
    else: 
        return print(f'Status code error: {r.status_code}')

In [7]:
# create all dfs using the function defined above
for i in range(0, 51):
    get_page(i)

In [8]:
players_df = pd.concat(players_list, ignore_index=True) #concat all dfs in the list to one df
players_df.to_csv("players.csv")

## Get Games from 2022 Season 

In [10]:
games_url = "https://www.balldontlie.io/api/v1/games"
games_list = [] # initialize empty list
def get_page_game(number):
    para = {"page" : number , "per_page" : 100, "seasons[]" : [2022]} # set parameters according to page number
    r = requests.get(games_url, params = para)
    r.status_code
    data = r.json()
    df = pd.DataFrame(data['data'])
    # clean to extract data from within inner dictionary
    df['home_team_id'] = df['home_team'].apply(lambda x: x.get('id') if isinstance(x, dict) else None)
    df['visitor_team_id'] = df['visitor_team'].apply(lambda x: x.get('id') if isinstance(x, dict) else None)
    df = df.drop(['home_team', 'visitor_team'], axis=1)
    # concat to the DataFrame
    games_list.append(df)
for i in range(0, 15):
    get_page_game(i)

In [11]:
games_df = pd.concat(games_list, ignore_index=True)
games_df.to_csv("games.csv")

## Get game and player stats

In [13]:
# put all game ids in a list
game_ids = games_df['id']
game_ids = game_ids.values.tolist()

stats_url = "https://www.balldontlie.io/api/v1/stats"
def get_game_stats(gameid):
    para = {"game_ids[]" : gameid}
    r = requests.get(stats_url, params = para)
    data = r.json()
    game_df = pd.DataFrame(data['data'])
    game_df['game_id'] = game_df['game'].apply(lambda x: x.get('id') if isinstance(x, dict) else None)
    game_df['player_id'] = game_df['player'].apply(lambda x: x.get('id') if isinstance(x, dict) else None)
    game_df['team_id'] = game_df['team'].apply(lambda x: x.get('id') if isinstance(x, dict) else None)
    game_df = game_df.drop(['game', 'player', 'team'], axis = 1)
    return game_df


method 2: use time.sleep between each game pull

In [29]:
all_stats = pd.DataFrame()

In [30]:
for id in game_ids:
    game_stat = get_game_stats(id)
    all_stats = pd.concat([all_stats, game_stat], ignore_index=True)
    time.sleep(1)

In [32]:
all_stats.to_csv("stats.csv", index = False)